#About this notebook

Here, we first download the relevant stock and sentiment data, clean it up a little and then merge them into a single dataframe.

We then run Granger on it and get the P-values and Best lag data, and then we store it as a csv on Google Drive.

To run the Logistic Regression test on only the stock data (without sentiment), we first generate signals with reference to the percentage change of each set of data (corresponding to each individual stock in our list): 1 if pctchange is +ve, otherwise 0. 
We then create a new dataframe with the original values as well as the lagged values, which we generate.

Our attempts to then run Logistic Regression on this data were unsuccessful, perhaps due to the fact that there was too much data included in the feed to the model.

In the [Sentiment Rerun](https://colab.research.google.com/drive/1n5QtOMVyAzOXJrPUl0uPFAi9R10PjAIc#scrollTo=l4W5R1wPAh5D) notebook, we managed to fix the issue and acquire the required results.

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd
import numpy as np
import statsmodels.tsa.stattools as ts
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

In [0]:
#Stock Data
id='10OYGX1iH_gFgISWsySJFqvjtH6V7S8fc'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_stocks_data = pd.read_csv('Filename.csv')
#IBM
id='1-2mTNh_iaTjEb9NK9OXC8wr43Bdx-LYS'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_IBM = pd.read_csv('Filename.csv')
#AAL
id='10Krqb5caDatIeA1-uCoGB_ygZqbX-sx7'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_AAL = pd.read_csv('Filename.csv')
#BAC
id='10KJhdi2LVYj0SRPpwzfViVG59wQuQG-L'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_BAC = pd.read_csv('Filename.csv')
#GM
id='10KpJmNOpH6HqNb9D3UGNVted7kejNMX-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_GM = pd.read_csv('Filename.csv')
#AXP
id='1-vPNJu3xDDotOhQg2r40PK7AYSdihLCh'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
df_AXP = pd.read_csv('Filename.csv')

In [0]:
df_IBM.drop('Unnamed: 0', axis=1, inplace=True)
df_AAL.drop('Unnamed: 0', axis=1, inplace=True)
df_GM.drop('Unnamed: 0', axis=1, inplace=True)
df_AXP.drop('Unnamed: 0', axis=1, inplace=True)
df_BAC.drop('Unnamed: 0', axis=1, inplace=True)

In [0]:
bras=[]
for i in range(len(df_stocks_data)):
  bras.append(i)
df_stocks_data.index=bras

In [0]:
import datetime
xy=df_stocks_data['date'].tolist()
datetimes=[]
for a in xy:
  date_time_obj = datetime.datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
  datetimes.append(date_time_obj)
df_stocks_data['Dates']=datetimes

In [0]:
df_stocks_data

In [0]:
dates_GM=df_GM['Date'].tolist()
times_GM=df_GM['Time'].tolist()
fintime_GM=[]
for a in range(len(times_GM)):
  fintime=dates_GM[a]+' '+times_GM[a]
  fintime_GM.append(fintime)
fintime_GM
df_GM['date']=fintime_GM
trialGM=pd.merge(df_stocks_data, df_GM, on='date')
trialGM.drop(['Date','Time','Close','Dates'], axis=1, inplace=True)
trialGM['GM_sentiment']=trialGM['sentiment scores']
trialGM.drop('sentiment scores', axis=1, inplace=True)

dates_AXP=df_AXP['Date'].tolist()
times_AXP=df_AXP['Time'].tolist()
fintime_AXP=[]
for a in range(len(times_AXP)):
  fintime=dates_AXP[a]+' '+times_AXP[a]
  fintime_AXP.append(fintime)
fintime_AXP
df_AXP['date']=fintime_AXP
trialAXP=pd.merge(trialGM, df_AXP, on='date')
trialAXP.drop(['Date','Time','Close'], axis=1, inplace=True)
trialAXP['AXP_sentiment']=trialAXP['sentiment scores']
trialAXP.drop('sentiment scores', axis=1, inplace=True)

dates_AAL=df_AAL['Date'].tolist()
times_AAL=df_AAL['Time'].tolist()
fintime_AAL=[]
for a in range(len(times_AAL)):
  fintime=dates_AAL[a]+' '+times_AAL[a]
  fintime_AAL.append(fintime)
fintime_AAL
df_AAL['date']=fintime_AAL
trialAAL=pd.merge(trialAXP, df_AAL, on='date')
trialAAL.drop(['Date','Time','Close'], axis=1, inplace=True)
trialAAL['AAL_sentiment']=trialAAL['sentiment scores']
trialAAL.drop('sentiment scores', axis=1, inplace=True)

dates_BAC=df_BAC['Date'].tolist()
times_BAC=df_BAC['Time'].tolist()
fintime_BAC=[]
for a in range(len(times_BAC)):
  fintime=dates_BAC[a]+' '+times_BAC[a]
  fintime_BAC.append(fintime)
fintime_BAC
df_BAC['date']=fintime_BAC
trialBAC=pd.merge(trialAAL, df_BAC, on='date')
trialBAC.drop(['Date','Time','Close'], axis=1, inplace=True)
trialBAC['BAC_sentiment']=trialBAC['sentiment scores']
trialBAC.drop('sentiment scores', axis=1, inplace=True)

In [0]:
finale=trialBAC

In [0]:
finale

,date,MSFT,AAPL,AMZN,GOOGL,ORCL,JPM,GS,AXP,RIO,VZ,SAP,UL,HSBC,V,BAC,T,JNJ,NVS,MRK,UNH,XOM,CVX,TOT,TM,F,NSANY,TSLA,GM,HOG,BA,DAL,LUV,UAL,AAL,SAVE,MCD,CMG,KMX,MUSA,CTB,SAH,GPC,H,LVS,MAR,PFE,PG,CHL,ABT,MDT,WMT,KO,PEP,BABA,NKE,DIS,NFLX,EROS,WWE,CNK,DELL,IBM,HPQ,FIT,HMI,TSM,USEG,PTR,SNP,UNP,SPCE,LMT,NOC,UPS,FDX,GM_sentiment,AXP_sentiment,AAL_sentiment,BAC_sentiment
0,2020-06-12 16:00:00,187.73,338.5300,2543.7600,1413.2600,51.850,99.8200,201.7800,101.6800,57.50,56.5100,130.130,54.7700,23.980,192.1900,24.7700,30.490,142.0300,84.340,76.290,284.870,47.1400,92.39,39.590,127.7301,6.4600,7.930,935.2000,27.9500,25.080,189.5100,30.4300,35.8700,39.6600,16.7400,18.8100,189.010,992.950,89.9900,109.60,26.170,28.795,85.56,55.8800,49.9000,92.2500,33.7500,115.620,34.9300,88.990,93.200,117.6800,45.5900,128.940,217.68,96.380,115.4750,418.0900,3.230,44.48,15.6500,46.900,121.930,16.2800,6.1100,10.4500,56.000,6.1200,34.310,44.130,166.090,14.9900,381.880,317.57,101.4700,132.730,0.65970,0.406000,0.070087,-0.011171
1,2020-06-12 15:55:00,188.05,339.1100,2549.8401,1417.2200,51.830,99.8700,201.1800,101.4500,57.46,56.6600,129.750,54.8200,23.975,192.2400,24.7100,30.530,142.1800,84.270,76.380,284.490,47.1550,92.45,39.580,127.9400,6.4400,7.850,935.7600,27.8200,25.025,189.1195,30.4100,35.7261,39.4650,16.6650,18.6850,188.911,992.630,89.9550,109.43,26.180,28.850,85.64,55.8900,49.8000,91.9633,33.7150,115.880,34.8900,89.150,93.095,117.8550,45.6050,129.180,217.46,96.460,115.5798,418.6900,3.200,44.51,15.6650,47.000,121.680,16.2700,6.1400,10.4500,55.910,NaN,34.300,44.030,166.210,14.9650,380.770,317.68,101.4300,132.243,0.00000,0.489200,-0.032073,-0.298450
2,2020-06-12 15:50:00,187.98,339.6700,2549.9124,1408.2500,51.885,99.7700,200.7600,101.3200,57.48,56.5850,129.950,54.7700,23.985,191.7314,24.7350,30.510,142.2500,84.320,76.500,284.020,47.1700,92.29,39.565,128.0200,6.4700,7.860,936.4800,27.8900,25.075,188.8000,30.3900,35.6700,39.1767,16.7400,18.6000,189.120,991.600,90.0561,109.34,26.080,28.810,85.85,55.6700,49.5800,91.5800,33.6800,115.970,34.8899,89.060,93.125,117.9800,45.6600,129.100,217.46,96.575,115.4200,416.5501,3.220,44.41,15.7089,46.980,121.680,16.3300,6.0950,10.4400,55.855,NaN,34.280,43.980,166.200,14.9100,380.645,317.30,101.4900,132.165,-0.07224,-0.068300,0.037230,0.246650
3,2020-06-12 15:45:00,187.30,338.4400,2544.0000,1400.4700,51.630,99.1700,199.9700,100.8000,57.28,56.3350,129.495,54.5500,23.895,190.9700,24.5801,30.395,141.6800,84.115,76.200,282.595,46.9600,91.96,39.430,127.6200,6.4450,7.820,931.4829,27.7800,24.980,187.3600,30.2810,35.5550,39.0250,16.6250,18.5700,188.240,988.710,89.6100,108.98,25.840,28.640,85.48,55.5300,49.3500,91.0918,33.5400,115.530,34.8800,88.610,92.800,117.5000,45.4682,128.450,217.13,96.130,114.8550,415.4700,3.180,44.06,15.6250,46.815,121.280,16.2750,6.1000,10.4200,55.690,6.1200,34.230,43.900,165.660,14.8850,378.510,315.61,101.0900,131.510,0.00000,0.220200,-0.038100,-0.097214
4,2020-06-12 15:40:00,187.04,337.8876,2539.0000,1395.0950,51.585,99.0500,199.8300,100.8300,57.24,56.2050,129.220,54.4950,23.850,190.5500,24.5600,30.335,141.4318,84.010,76.140,282.260,46.8350,91.88,39.400,127.5201,6.4450,7.810,929.7277,27.7500,24.950,187.1500,30.2550,35.4900,39.1010,16.5050,18.6343,188.050,986.470,89.5300,108.59,25.805,28.650,85.37,55.6000,49.2200,90.8850,33.4750,115.430,34.8400,88.435,92.640,117.4100,45.4000,128.280,216.83,96.130,114.8400,415.0733,3.170,43.99,15.6350,46.720,121.030,16.2550,6.0850,10.3800,55.650,NaN,34.220,43.920,165.250,14.8867,378.480,315.60,100.9777,131.080,0.11315,0.000000,0.016000,-0.176133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2020-06-08 10:00:00,185.12,329.1400,2496.1200,1430.4100,53.990,113.4300,222.0000,113.2600,58.05,57.3900,135.000,56.2100,

In [0]:
finale.index=finale['date']
finale.drop('date',axis=1, inplace=True)

In [0]:
#RUNNING THE GRANGER CAUSALITY TESTS ON THE DATAFRME WITH THE SENTIMENT DATA
Masterfin1=pd.DataFrame()
stox1=[]
stox2=[]
min_pvalue=[]
best_lag=[]
for data1 in list(finale.columns):
  for data2 in list(finale.columns):
    if data1==data2:
      continue
    data_1=finale[data1].tolist()
    data_2=finaleBAC[data2].tolist()
    granger_data=list(zip(data_1,data_2))
    try:
      granger_result=ts.grangercausalitytests(granger_data, maxlag=40, verbose=False)
    except Exception as e:
      print('Missing Data Error: '+str(e))
      print(': '+data1)
      print(': '+data2)
      continue
    pvalues=[]
    for i in range (40):
      ad=i+1
      a1=granger_result[ad]
      a1_=a1[0]
      x_params_1=a1_['params_ftest']
      x_likelihhod_ratio_1=a1_['lrtest']
      x_SSR_chi2_1=a1_['ssr_chi2test']
      pvalue=((x_params_1[1]+x_likelihhod_ratio_1[1]+x_SSR_chi2_1[1])/3) 
      pvalues.append(pvalue)
    min_pval=min(pvalues)
    min_index=0
    for i in range(len(pvalues)):
      if (pvalues[i]==min_pval):
        min_index=i+1
    min_pvalue.append(min_pval)
    best_lag.append(min_index)
    stox1.append(data1)
    stox2.append(data2)
Masterfin1['Param1']=stox1
Masterfin1['Param2']=stox2
Masterfin1['Min. p']=min_pvalue
Masterfin1['Best lag']=best_lag

Missing Data Error: exog contains inf or nans
: MSFT
: TM
Missing Data Error: exog contains inf or nans
: MSFT
: NSANY
Missing Data Error: exog contains inf or nans
: MSFT
: CMG
Missing Data Error: exog contains inf or nans
: MSFT
: MUSA
Missing Data Error: exog contains inf or nans
: MSFT
: CTB
Missing Data Error: exog contains inf or nans
: MSFT
: SAH
Missing Data Error: exog contains inf or nans
: MSFT
: HMI
Missing Data Error: exog contains inf or nans
: MSFT
: USEG
Missing Data Error: exog contains inf or nans
: MSFT
: PTR
Missing Data Error: exog contains inf or nans
: MSFT
: SNP
Missing Data Error: exog contains inf or nans
: AAPL
: TM
Missing Data Error: exog contains inf or nans
: AAPL
: NSANY
Missing Data Error: exog contains inf or nans
: AAPL
: CMG
Missing Data Error: exog contains inf or nans
: AAPL
: MUSA
Missing Data Error: exog contains inf or nans
: AAPL
: CTB
Missing Data Error: exog contains inf or nans
: AAPL
: SAH
Missing Data Error: exog contains inf or nans
: AAP

In [0]:
Masterfin1

,Param1,Param2,Min. p,Best lag
0,MSFT,AAPL,2.080822e-04,6
1,MSFT,AMZN,2.081506e-02,19
2,MSFT,GOOGL,1.151086e-07,16
3,MSFT,ORCL,1.369205e-02,2
4,MSFT,JPM,1.745764e-02,25
...,...,...,...,...
4687,BAC_sentiment,UPS,1.609280e-05,1
4688,BAC_sentiment,FDX,3.072575e-05,1
4689,BAC_sentiment,GM_sentiment,1.874867e-01,40
4690,BAC_sentiment,AXP_sentiment,5.143507e-02,40


In [0]:
from google.colab import drive
drive.mount('drive')
Masterfin1.to_csv('INTRADAYSTOXDATAWITHSENTIMENT.csv')
!cp INTRADAYSTOXDATAWITHSENTIMENT.csv "drive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
#df_stocks_data.index=df_stocks_data['date']
df_stocks_data.drop('date',axis=1, inplace=True)
df_stocks_data

,MSFT,AAPL,AMZN,GOOGL,ORCL,JPM,GS,AXP,RIO,VZ,SAP,UL,HSBC,V,BAC,T,JNJ,NVS,MRK,UNH,XOM,CVX,TOT,TM,F,NSANY,TSLA,GM,HOG,BA,DAL,LUV,UAL,AAL,SAVE,MCD,CMG,KMX,MUSA,CTB,SAH,GPC,H,LVS,MAR,PFE,PG,CHL,ABT,MDT,WMT,KO,PEP,BABA,NKE,DIS,NFLX,EROS,WWE,CNK,DELL,IBM,HPQ,FIT,HMI,TSM,USEG,PTR,SNP,UNP,SPCE,LMT,NOC,UPS,FDX
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-12 16:00:00,187.7300,338.5300,2543.7600,1413.260,51.850,99.820,201.78,101.6800,57.50,56.510,130.130,54.770,23.980,192.1900,24.7700,30.490,142.0300,84.3400,76.2900,284.870,47.140,92.3900,39.5900,127.7301,6.4600,7.930,935.2000,27.950,25.080,189.5100,30.4300,35.8700,39.6600,16.7400,18.8100,189.010,992.9500,89.9900,109.60,26.170,28.795,85.560,55.880,49.900,92.2500,33.750,115.620,34.9300,88.9900,93.200,117.680,45.5900,128.9400,217.680,96.380,115.4750,418.0900,3.2300,44.48,15.6500,46.900,121.9300,16.2800,6.1100,10.45,56.000,6.1200,34.310,44.1300,166.0900,14.9900,381.880,317.570,101.4700,132.7300
2020-06-12 15:55:00,188.0500,339.1100,2549.8401,1417.220,51.830,99.870,201.18,101.4500,57.46,56.660,129.750,54.820,23.975,192.2400,24.7100,30.530,142.1800,84.2700,76.3800,284.490,47.155,92.4500,39.5800,127.9400,6.4400,7.850,935.7600,27.820,25.025,189.1195,30.4100,35.7261,39.4650,16.6650,18.6850,188.911,992.6300,89.9550,109.43,26.180,28.850,85.640,55.890,49.800,91.9633,33.715,115.880,34.8900,89.1500,93.095,117.855,45.6050,129.1800,217.460,96.460,115.5798,418.6900,3.2000,44.51,15.6650,47.000,121.6800,16.2700,6.1400,10.45,55.910,NaN,34.300,44.0300,166.2100,14.9650,380.770,317.680,101.4300,132.2430
2020-06-12 15:50:00,187.9800,339.6700,2549.9124,1408.250,51.885,99.770,200.76,101.3200,57.48,56.585,129.950,54.770,23.985,191.7314,24.7350,30.510,142.2500,84.3200,76.5000,284.020,47.170,92.2900,39.5650,128.0200,6.4700,7.860,936.4800,27.890,25.075,188.8000,30.3900,35.6700,39.1767,16.7400,18.6000,189.120,991.6000,90.0561,109.34,26.080,28.810,85.850,55.670,49.580,91.5800,33.680,115.970,34.8899,89.0600,93.125,117.980,45.6600,129.1000,217.460,96.575,115.4200,416.5501,3.2200,44.41,15.7089,46.980,121.6800,16.3300,6.0950,10.44,55.855,NaN,34.280,43.9800,166.2000,14.9100,380.645,317.300,101.4900,132.1650
2020-06-12 15:45:00,187.3000,338.4400,2544.0000,1400.470,51.630,99.170,199.97,100.8000,57.28,56.335,129.495,54.550,23.895,190.9700,24.5801,30.395,141.6800,84.1150,76.2000,282.595,46.960,91.9600,39.4300,127.6200,6.4450,7.820,931.4829,27.780,24.980,187.3600,30.2810,35.5550,39.0250,16.6250,18.5700,188.240,988.7100,89.6100,108.98,25.840,28.640,85.480,55.530,49.350,91.0918,33.540,115.530,34.8800,88.6100,92.800,117.500,45.4682,128.4500,217.130,96.130,114.8550,415.4700,3.1800,44.06,15.6250,46.815,121.2800,16.2750,6.1000,10.42,55.690,6.1200,34.230,43.9000,165.6600,14.8850,378.510,315.610,101.0900,131.5100
2020-06-12 15:40:00,187.0400,337.8876,2539.0000,1395.095,51.585,99.050,199.83,100.8300,57.24,56.205,129.220,54.495,23.850,190.5500,24.5600,30.335,141.4318,84.0100,76.1400,282.260,46.835,91.8800,39.4000,127.5201,6.4450,7.810,929.7277,27.750,24.950,187.1500,30.2550,35.4900,39.1010,16.5050,18.6343,188.050,986.4700,89.5300,108.59,25.805,28.650,85.370,55.600,49.220,90.8850,33.475,115.430,34.8400,88.4350,92.640,117.410,45.4000,128.2800,216.830,96.130,114.8400,415.0733,3.1700,43.99,15.6350,46.720,121.0300,16.2550,6.0850,10.38,55.650,NaN,34.220,43.9200,165.2500,14.8867,378.480,315.600,100.9777,131.0800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22 09:55:00,184.2300,317.2000,2460.3950,1404.595,52.400,89.260,179.22,88.8200,51.66,53.810,116.040,50.500,23.295,191.0800,22.6150,29.595,144.8300,84.1550,76.3400,285.280,43.830,90.7900,35.7200,117.5499,5.6076,7.000,827.9126,25.670,23.850,138.0500,22.8000,29.0623,25.6100,9.8950,10.1900,184

In [0]:
#WITH ONLY STOCK DATA
df_stocks_data.fillna(method='ffill', inplace=True)
df_stocks_data3=df_stocks_data
dfsh=[]
for i in range(len(df_stocks_data3)):
  dfsh.append(i)
df_stocks_data3.index=dfsh
for stock in list(df_stocks_data3.columns):
  signal = np.where(df_stocks_data3[stock] >= df_stocks_data3[stock].shift(), 1, 0)
  # percentage_change=df_stocks_data3[stock].pct_change().to_list()
  # signal=[]
  # for x in percentage_change:
  #   if (x>=0.0):
  #     signal.append(1)
  #   else:
  #     signal.append(0)
  signal=pd.Series(signal)
  df_stocks_data3[stock+'_00sig'] = signal
  df_stocks_data3[stock+'_01sig'] = signal.shift(1)
  df_stocks_data3[stock+'_02sig'] = signal.shift(2)
  df_stocks_data3[stock+'_03sig'] = signal.shift(3)
  df_stocks_data3[stock+'_04sig'] = signal.shift(4)
  df_stocks_data3[stock+'_05sig'] = signal.shift(5)
  df_stocks_data3[stock+'_06sig'] = signal.shift(6)
  df_stocks_data3[stock+'_07sig'] = signal.shift(7)
  df_stocks_data3[stock+'_08sig'] = signal.shift(8)
  df_stocks_data3[stock+'_09sig'] = signal.shift(9)
  df_stocks_data3[stock+'_10sig'] = signal.shift(10)
  df_stocks_data3[stock+'_11sig'] = signal.shift(11)
  df_stocks_data3[stock+'_12sig'] = signal.shift(12)
  df_stocks_data3[stock+'_13sig'] = signal.shift(13)
  df_stocks_data3[stock+'_14sig'] = signal.shift(14)
  df_stocks_data3[stock+'_15sig'] = signal.shift(15)
df_stocks_data3.dropna(inplace=True)

In [0]:
df_stocks_data3

,MSFT,AAPL,AMZN,GOOGL,ORCL,JPM,GS,AXP,RIO,VZ,SAP,UL,HSBC,V,BAC,T,JNJ,NVS,MRK,UNH,XOM,CVX,TOT,TM,F,NSANY,TSLA,GM,HOG,BA,DAL,LUV,UAL,AAL,SAVE,MCD,CMG,KMX,MUSA,CTB,...,NOC_08sig,NOC_09sig,NOC_10sig,NOC_11sig,NOC_12sig,NOC_13sig,NOC_14sig,NOC_15sig,UPS_00sig,UPS_01sig,UPS_02sig,UPS_03sig,UPS_04sig,UPS_05sig,UPS_06sig,UPS_07sig,UPS_08sig,UPS_09sig,UPS_10sig,UPS_11sig,UPS_12sig,UPS_13sig,UPS_14sig,UPS_15sig,FDX_00sig,FDX_01sig,FDX_02sig,FDX_03sig,FDX_04sig,FDX_05sig,FDX_06sig,FDX_07sig,FDX_08sig,FDX_09sig,FDX_10sig,FDX_11sig,FDX_12sig,FDX_13sig,FDX_14sig,FDX_15sig
15,186.4400,337.2300,2523.6201,1392.290,51.380,98.1600,197.900,99.5300,57.02,56.140,128.760,54.490,23.740,189.6700,24.2998,30.2300,140.946,83.7600,75.7600,279.870,46.2900,90.6850,39.2600,126.9400,6.3550,7.850,922.8700,27.410,24.58,182.7700,29.7150,35.4150,38.1701,16.3650,18.2175,187.130,975.6200,88.6600,107.445,25.530,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
16,186.6200,337.8610,2532.1206,1391.920,51.390,98.4499,198.435,99.7000,57.11,56.150,128.740,54.490,23.730,189.9700,24.3750,30.2761,141.090,83.8200,75.8199,281.050,46.3950,90.9100,39.2650,126.9900,6.3750,7.850,924.3100,27.485,24.58,183.8878,29.6900,35.4000,38.0800,16.3901,18.1650,187.495,977.5800,88.8500,106.835,25.500,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
17,187.0900,338.1550,2536.4199,1394.100,51.490,98.6700,199.100,99.9000,57.20,56.180,129.015,54.460,23.770,190.2319,24.4800,30.3550,141.170,83.9100,75.9416,281.770,46.5400,91.1600,39.3200,127.1700,6.3900,7.835,927.6899,27.570,24.69,185.4600,29.7801,35.4385,38.1400,16.4550,18.2700,187.730,980.0000,89.1300,107.245,25.555,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
18,186.4901,336.8595,2531.5801,1392.050,51.350,98.4650,198.440,99.6400,57.08,56.090,128.470,54.370,23.740,189.8600,24.3978,30.2600,140.970,83.6850,75.8350,280.490,46.3250,90.9500,39.1100,126.9300,6.3728,7.835,925.9792,27.460,24.66,184.1500,29.6000,35.3300,37.8300,16.3705,18.1137,187.080,975.1500,88.9000,106.870,25.430,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
19,186.0500,336.0761,2522.1399,1388.110,51.230,98.2700,198.010,99.1700,56.93,56.020,128.030,54.330,23.680,189.3301,24.3150,30.1950,140.690,83.5750,75.7800,280.210,46.2281,90.8550,39.0800,126.7700,6.3550,7.835,922.4070,27.450,24.60,183.3800,29.5050,35.2150,37.6800,16.3150,17.9700,186.810,969.2900,88.7400,106.735,25.420,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,184.2300,317.2000,2460.3950,1404.595,52.400,89.2600,179.220,88.8200,51.66,53.810,116.040,50.500,23.295,191.0800,22.6150,29.5950,144.830,84.1550,76.3400,285.280,43.8300,90.7900,35.7200,117.5499,5.6076,7.000,827.9126,25.670,23.85,138.0500,22.8000,29.0623,25.6100,9.8950,10.1900,184.320,1026.5601,80.7000,114.960,24.000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1141,183.9400,316.7200,2454.7300,1402.260,52.190,89.6400,179.800,89.3900,51.56,53.800,115.940,50.430,23.330,191.2300,22.7450,29.6150,144.820,84.0700,76.4400,285.360,43.7800,90.5238,35.7800,117.5200,5.6050,7.000,823.1580,25.63

In [0]:
#1: BAC
df1=df_stocks_data3.copy()
target_ar=df1['BAC_00sig'].tolist()
df1.drop('BAC_00sig', axis=1, inplace=True)
df1.to_numpy
sm_model = sm.Logit(target_ar, df1)
result = sm_model.fit(method='bfgs')
#clf=LogisticRegression(random_state=0).fit(df1, target_ar)
Mastermast=pd.DataFrame()
#Mastermast['R-squared']=sm_model.rsquared


/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1736: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1736: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/usr/local/lib/python3.6/dist-packages/statsmodels/discrete/discrete_model.py:1789: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


         Current function value: 0.693147
         Iterations: 0
         Function evaluations: 16
         Gradient evaluations: 4


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


AttributeError: ignored

In [0]:
result.summary()

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 1130
Model:                          Logit   Df Residuals:                        0
Method:                           MLE   Df Model:                         1129
Date:                Mon, 15 Jun 2020   Pseudo R-squ.:               -0.001530
Time:                        18:39:50   Log-Likelihood:                -783.26
converged:                      False   LL-Null:                       -782.06
Covariance Type:            nonrobust   LLR p-value:                     1.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
MSFT                 0        nan        nan        nan         nan         nan
AAPL                 0        nan        nan        nan         nan         nan
AMZN                 0        nan        nan        nan         nan         nan
GOOGL                0        nan        nan        nan         nan         nan
ORCL                 0        nan        nan        nan         nan         nan
JPM                  0        nan        nan        nan         nan         nan
GS                   0        nan        nan        nan         nan         nan
AXP                  0        nan        nan        nan         nan         nan
RIO                  0        nan        nan        nan         nan         nan
VZ                   0        nan        nan        nan         nan         nan
SAP                  0        nan        nan        nan         nan         nan
UL                   0        nan        nan        nan         nan         nan
HSBC                 0        nan        nan        nan         nan         nan
V                    0        nan        nan        nan         nan         nan
BAC                  0        nan        nan        nan         nan         nan
T                    0        nan        nan        nan         nan         nan
JNJ                  0        nan        nan        nan         nan         nan
NVS                  0        nan        nan        nan         nan         nan
MRK                  0        nan        nan        nan         nan         nan
UNH                  0        nan        nan        nan         nan         nan
XOM                  0        nan        nan        nan         nan         nan
CVX                  0        nan        nan        nan         nan         nan
TOT                  0        nan        nan        nan         nan         nan
TM                   0        nan        nan        nan         nan         nan
F                    0        nan        nan        nan         nan         nan
NSANY                0        nan        nan        nan         nan         nan
TSLA                 0        nan        nan        nan         nan         nan
GM                   0        nan        nan        nan         nan         nan
HOG                  0        nan        nan        nan         nan         nan
BA                   0        nan        nan        nan         nan         nan
DAL                  0        nan        nan        nan         nan         nan
LUV                  0        nan        nan        nan         nan         nan
UAL                  0        nan        nan        nan         nan         nan
AAL                  0        nan        nan        nan         nan         nan
SAVE                 0        nan        nan        nan         nan         nan
MCD                  0        nan        nan        nan         nan         nan
CMG                  0        nan        nan        nan         nan         nan
KMX                  0        nan        

In [0]:
df

In [0]:
df1.describe()

,MSFT,AAPL,AMZN,GOOGL,ORCL,JPM,GS,AXP,RIO,VZ,SAP,UL,HSBC,V,BAC,T,JNJ,NVS,MRK,UNH,XOM,CVX,TOT,TM,F,NSANY,TSLA,GM,HOG,BA,DAL,LUV,UAL,AAL,SAVE,MCD,CMG,KMX,MUSA,CTB,...,T_11sig_07sig_08sig,T_11sig_07sig_09sig,T_11sig_07sig_10sig,T_11sig_07sig_11sig,T_11sig_07sig_12sig,T_11sig_07sig_13sig,T_11sig_07sig_14sig,T_11sig_07sig_15sig,T_11sig_08sig_00sig,T_11sig_08sig_01sig,T_11sig_08sig_02sig,T_11sig_08sig_03sig,T_11sig_08sig_04sig,T_11sig_08sig_05sig,T_11sig_08sig_06sig,T_11sig_08sig_07sig,T_11sig_08sig_08sig,T_11sig_08sig_09sig,T_11sig_08sig_10sig,T_11sig_08sig_11sig,T_11sig_08sig_12sig,T_11sig_08sig_13sig,T_11sig_08sig_14sig,T_11sig_08sig_15sig,T_11sig_09sig_00sig,T_11sig_09sig_01sig,T_11sig_09sig_02sig,T_11sig_09sig_03sig,T_11sig_09sig_04sig,T_11sig_09sig_05sig,T_11sig_09sig_06sig,T_11sig_09sig_07sig,T_11sig_09sig_08sig,T_11sig_09sig_09sig,T_11sig_09sig_10sig,T_11sig_09sig_11sig,T_11sig_09sig_12sig,T_11sig_09sig_13sig,T_11sig_09sig_14sig,T_11sig_09sig_15sig
count,431.000000,431.000000,431.00000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,...,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000,431.000000
mean,185.435444,328.253670,2492.48797,1433.368065,53.395921,103.492628,207.732763,103.129300,55.565499,56.523970,130.755064,54.723159,24.810407,195.876094,26.067767,31.618779,146.494505,86.080624,80.361832,302.774595,48.930253,95.951107,40.160697,127.894616,6.455265,8.061405,889.860614,28.404900,24.891149,179.082705,29.570972,35.680265,35.853852,14.572238,17.690724,192.228312,1034.099701,91.471754,114.223423,26.907031,...,0.505800,0.519722,0.517401,0.491879,0.515081,0.519722,0.526682,0.487239,0.515081,0.531323,0.547564,0.519722,0.503480,0.505800,0.542923,0.505800,0.519722,0.517401,0.491879,0.515081,0.519722,0.526682,0.487239,0.505800,0.531323,0.547564,0.519722,0.503480,0.505800,0.542923,0.505800,0.519722,0.517401,0.491879,0.515081,0.519722,0.526682,0.487239,0.505800,0.480278
std,3.811012,10.364445,75.12181,15.409015,0.722943,6.393104,10.728258,6.552711,2.197701,1.153576,5.909892,1.739852,1.219612,3.059418,1.708576,0.903588,1.745161,1.196776,2.093141,7.519130,3.335125,4.173910,2.526160,3.272946,0.648289,0.504886,60.949805,1.582777,1.480655,30.521552,4.195465,3.083439,6.932576,3.616853,4.631123,5.412937,23.302296,4.262651,1.994331,1.423862,...,0.500547,0.500192,0.500278,0.500515,0.500353,0.500192,0.499868,0.500418,0.500353,0.499598,0.498311,0.500192,0.500569,0.500547,0.498733,0.500547,0.500192,0.500278,0.500515,0.500353,0.500192,0.499868,0.500418,0.500547,0.499598,0.498311,0.500192,0.500569,0.500547,0.498733,0.500547,0.500192,0.500278,0.500515,0.500353,0.500192,0.499868,0.500418,0.500547,0.500192
min,176.640000,313.240500,2331.28000,1386.010000,51.020000,89.060000,178.850000,88.420000,51.320000,53.705000,115.445000,50.300000,22.770000,188.530000,22.605000,29.530000,140.580000,83.330000,75.650000,279.350000,43.730000,89.400000,35.515000,117.430000,5.559000,6.990000,785.600000,25.495000,21.055000,136.130100,22.373000,28.615000,24.915000,9.685000,9.965000,183.450000,965.000000,80.320000,106.050000,23.910000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000